# Backtest Results

## Apply Model to all available Ground-truth data

In [1]:
import argparse
import json
import logging
import os
from traceback import print_exc
from tqdm import tqdm

FORMAT = '%(asctime)-15s %(levelname)-5s %(message)s'
logging.basicConfig(format=FORMAT, level=logging.INFO)

#import numpy as np
import pandas as pd

from research_lib.utils.data_access_utils import S3AccessUtils
from research_lib.utils.datetime_utils import add_days
from filter_optimization.filter_optimization_task import generate_global_optimum_filter, _add_date_hour_columns
from report_generation.report_generator import generate_ts_data, SamplingFilter
from weight_estimation.weight_estimator import WeightEstimator
from weight_estimation.utils import CameraMetadata



#s3 = S3AccessUtils('/tmp', json.load(open(os.environ['AWS_CREDENTIALS'])))
s3 = S3AccessUtils('/tmp')
ROOT_DIR = '/tmp/batches'
AKPD_SCORE_CUTOFF = 0.99
weight_model_f = '/root/alok/repos/internal-tools/biomass-backtest/.data/nn_epoch_798_v2.pb'
kf_model_f = '/root/alok/repos/internal-tools/biomass-backtest/.data//kf_predictor_v2.pb'


def estimate_weight_task(df: pd.DataFrame) -> pd.DataFrame:
    """Generates individual weight, length, and k-factor estimates for input dataset."""

    logging.info(f"estimate_weight_task data: {len(df)} rows")

    for col in ['annotation', 'camera_metadata']:
        if type(df[col].iloc[0]) != dict:
            df[col] = df[col].apply(lambda x: json.loads(x.replace("'", '"')))

    weight_estimator = WeightEstimator(weight_model_f, kf_model_f)
    weights, lengths, kfs = [], [], []
    count = 0
    for idx, row in tqdm(df.iterrows()):
#         if count % 500 == 0:
#             print('{} - Percentage completion: {}%'.format(count, round(100 * count / df.shape[0], 2)))
        count += 1
        annotation = row.annotation
        camera_metadata = row.camera_metadata

        camera_metadata_obj = CameraMetadata(
            focal_length=camera_metadata['focalLength'],
            focal_length_pixel=camera_metadata['focalLengthPixel'],
            baseline_m=camera_metadata['baseline'],
            pixel_count_width=camera_metadata['pixelCountWidth'],
            pixel_count_height=camera_metadata['pixelCountHeight'],
            image_sensor_width=camera_metadata['imageSensorWidth'],
            image_sensor_height=camera_metadata['imageSensorHeight']
        )

        weight, length, kf = weight_estimator.predict(annotation, camera_metadata_obj)
        weights.append(weight)
        lengths.append(length)
        kfs.append(kf)

    df['estimated_weight_g'] = weights
    df['estimated_length_mm'] = lengths
    df['estimated_k_factor'] = kfs
    return df


def compute_filter_task(df: pd.DataFrame, last_feeding_date: str) -> dict:
    """Compute optimal filter for this dataset."""

    df = _add_date_hour_columns(df)
    end_date = add_days(last_feeding_date, 1) if last_feeding_date else df.date.iloc[-1]
    start_date = add_days(end_date, -14)
    df = df[(df.date >= start_date) & (df.date <= end_date)]

    print(f"compute_filter_task {start_date}-{end_date} -> {len(df)} records")
    sampling_filter_obj = generate_global_optimum_filter(df)
    sampling_filter = {
        'best_start_hr': sampling_filter_obj.start_hour,
        'best_end_hr': sampling_filter_obj.end_hour,
        'best_kf_cutoff': sampling_filter_obj.kf_cutoff
    }
    # sampling_filter = {
    #     'best_start_hr': 7,
    #     'best_end_hr': 15,
    #     'best_kf_cutoff': 0.0
    # }

    return sampling_filter


def generate_report_task(df, sampling_filter):
    sampling_filter_obj = SamplingFilter(
        start_hour=sampling_filter['best_start_hr'],
        end_hour=sampling_filter['best_end_hr'],
        kf_cutoff=sampling_filter['best_kf_cutoff'],
        akpd_score_cutoff=AKPD_SCORE_CUTOFF
    )

    df = _add_date_hour_columns(df)
    ts_data = pd.DataFrame(generate_ts_data(df, sampling_filter_obj))
    return ts_data


def main(cohort_names, batch_name, use_dw_values):

    for cohort_name in cohort_names:
        print('Processing:{}'.format(cohort_name))

        try:
            s3_dir = os.path.join(
                'https://aquabyte-images-adhoc.s3-eu-west-1.amazonaws.com/alok/production_datasets',
                cohort_name
            )

            ground_truth_metadata_url = os.path.join(s3_dir, 'ground_truth_metadata.json')
            ground_truth_key_base = os.path.join(batch_name, cohort_name, 'ground_truth_metadata.json')
            ground_truth_f = os.path.join(ROOT_DIR, ground_truth_key_base)
            try:
                logging.info(f"Loading ground_truth_metadata_url: {ground_truth_metadata_url}")
                s3.download_from_url(ground_truth_metadata_url, custom_location=ground_truth_f)
                s3.s3_client.upload_file(ground_truth_f, 'aquabyte-images-adhoc',
                                         os.path.join('backtest', ground_truth_key_base))
            except:
                logging.info('Couldnt download groundtruth for cohort: {}'.format(cohort_name))
                continue

            # generate biomass computations
            data_url = os.path.join(s3_dir, 'annotation_dataset.csv')
            logging.info(f"Loading {data_url}")
            data_f, _, _= s3.download_from_url(data_url)
            df = pd.read_csv(data_f)
            df = df[df.akpd_score >= AKPD_SCORE_CUTOFF]

            if not use_dw_values:
                biomass_computations_df = estimate_weight_task(df)
            else:
                biomass_computations_df = df

            out_key_base = os.path.join(batch_name, cohort_name, 'biomass_computations.csv')
            out_f = os.path.join(ROOT_DIR, out_key_base)
            biomass_computations_df.to_csv(out_f)
            logging.info(f"estimate_weight_task -> {out_key_base}")
            s3.s3_client.upload_file(out_f, 'aquabyte-images-adhoc',
                                     os.path.join('backtest', out_key_base))

            # generate optimal filters
            ground_truth_metadata = json.load(open(ground_truth_f))
            last_feeding_date = ground_truth_metadata['last_feeding_date']
            sampling_filter = compute_filter_task(biomass_computations_df, last_feeding_date)
            logging.info(f"compute_filter_task -> {sampling_filter}")

            out_key_base = os.path.join(batch_name, cohort_name, 'sampling_filter.json')
            out_f = os.path.join(ROOT_DIR, out_key_base)
            logging.info(f"compute_filter_task -> {out_key_base}")
            json.dump(sampling_filter, open(out_f, 'w'))
            s3.s3_client.upload_file(out_f, 'aquabyte-images-adhoc',
                                     os.path.join('backtest', out_key_base))

            # generate preliminary report
            ts_data = generate_report_task(biomass_computations_df, sampling_filter)

            out_key_base = os.path.join(ROOT_DIR, batch_name, cohort_name, 'report.csv')
            logging.info(f"generate_report_task -> {out_key_base}")

            out_f = os.path.join(ROOT_DIR, out_key_base)
            ts_data.to_csv(out_f)
            s3.s3_client.upload_file(out_f, 'aquabyte-images-adhoc',
                                     os.path.join('backtest', out_key_base))
        except:
            logging.info('Got error for {}, skipping...'.format(cohort_name))
            print_exc()



2021-01-15 15:07:32,861 INFO  Found credentials in shared credentials file: ~/.aws/credentials


In [2]:
BATCH_NAME = 'sidtest3'
p = 'alok/production_datasets/'
gt = '/ground_truth_metadata.json'

cohort_names = [x.replace(p, '').replace(gt, '') for x in s3.get_matching_s3_keys('aquabyte-images-adhoc', p)
 if x.endswith(gt)]
cohort_names

alok/production_datasets/


['aplavika_pen_id_95_2020-07-10_2020-07-26',
 'bolaks_pen_id_88_2020-02-28_2020-03-10',
 'dale_pen_id_143_2020-10-07_2020-10-21',
 'djubawik_pen_id_153_2020-11-10_2020-11-26',
 'eldviktaren_pen_id_164_2020-09-21_2020-10-08',
 'habranden_pen_id_100_2020-08-10_2020-08-31',
 'kjeppevikholmen_pen_id_5_2019-06-18_2019-07-02',
 'langoy_pen_id_108_2020-05-07_2020-05-17',
 'leivsethamran_pen_id_165_2020-10-18_2020-11-13',
 'movikodden_pen_id_114_2020-11-03_2020-11-25',
 'movikodden_pen_id_167_2020-10-13_2020-10-30',
 'seglberget_pen_id_66_2020-05-13_2020-06-13',
 'silda_pen_id_86_2020-07-02_2020-07-19',
 'slapoya_pen_id_116_2020-10-18_2020-11-08',
 'tittelsnes_pen_id_37_2020-06-10_2020-06-24',
 'varholmen_pen_id_131_2020-08-15_2020-08-30',
 'varholmen_pen_id_151_2020-10-02_2020-10-17',
 'varholmen_pen_id_186_2020-10-18_2020-11-02',
 'vikane_pen_id_60_2020-08-10_2020-08-30']

In [3]:
main(
    cohort_names[5:],
    batch_name = BATCH_NAME,
    use_dw_values = False)

2021-01-15 15:08:13,648 INFO  Loading ground_truth_metadata_url: https://aquabyte-images-adhoc.s3-eu-west-1.amazonaws.com/alok/production_datasets/habranden_pen_id_100_2020-08-10_2020-08-31/ground_truth_metadata.json


Processing:habranden_pen_id_100_2020-08-10_2020-08-31


2021-01-15 15:08:14,343 INFO  Loading https://aquabyte-images-adhoc.s3-eu-west-1.amazonaws.com/alok/production_datasets/habranden_pen_id_100_2020-08-10_2020-08-31/annotation_dataset.csv
2021-01-15 15:08:16,382 INFO  estimate_weight_task data: 36357 rows
36357it [00:37, 980.07it/s] 
2021-01-15 15:09:00,648 INFO  estimate_weight_task -> sidtest3/habranden_pen_id_100_2020-08-10_2020-08-31/biomass_computations.csv


compute_filter_task 2020-08-11-2020-08-25 -> 27577 records
Performing coarse grid search...
Start hour: 0, End hour: 13, KF cutoff: 0.9
Insufficient data to compute distribution consistency!
Insufficient data to compute distribution consistency!
Start hour: 0, End hour: 13, KF cutoff: 0.9500000000000001
Insufficient data to compute distribution consistency!
Insufficient data to compute distribution consistency!
Start hour: 0, End hour: 13, KF cutoff: 1.0
Insufficient data to compute distribution consistency!
Insufficient data to compute distribution consistency!
Start hour: 0, End hour: 13, KF cutoff: 1.0500000000000003
Insufficient data to compute distribution consistency!
Insufficient data to compute distribution consistency!
Start hour: 0, End hour: 13, KF cutoff: 1.1
Insufficient data to compute distribution consistency!
Insufficient data to compute distribution consistency!
Start hour: 0, End hour: 13, KF cutoff: 1.1500000000000004
Insufficient data to compute distribution consist

Insufficient data to compute distribution consistency!
Start hour: 6, End hour: 13, KF cutoff: 1.1
Insufficient data to compute distribution consistency!
Insufficient data to compute distribution consistency!
Start hour: 6, End hour: 13, KF cutoff: 1.1500000000000004
Insufficient data to compute distribution consistency!
Insufficient data to compute distribution consistency!
Start hour: 6, End hour: 13, KF cutoff: 1.2000000000000002
Insufficient data to compute distribution consistency!
Insufficient data to compute distribution consistency!
Start hour: 6, End hour: 13, KF cutoff: 1.2500000000000004
Insufficient data to compute distribution consistency!
Insufficient data to compute distribution consistency!
Start hour: 6, End hour: 13, KF cutoff: 1.3000000000000003
Insufficient data to compute distribution consistency!
Insufficient data to compute distribution consistency!
Start hour: 6, End hour: 13, KF cutoff: 1.3500000000000005
Insufficient data to compute distribution consistency!
I

...

Start hour: 0.0, End hour: 23.0, KF cutoff: 1.109999999999999
Insufficient data to compute distribution consistency!
Start hour: 0.0, End hour: 23.0, KF cutoff: 1.1149999999999989
Insufficient data to compute distribution consistency!
Start hour: 0.0, End hour: 23.0, KF cutoff: 1.1199999999999988
Insufficient data to compute distribution consistency!
Start hour: 0.0, End hour: 23.0, KF cutoff: 1.1249999999999987
Insufficient data to compute distribution consistency!
Start hour: 0.0, End hour: 23.0, KF cutoff: 1.1299999999999986
Insufficient data to compute distribution consistency!
Start hour: 0.0, End hour: 23.0, KF cutoff: 1.1349999999999985
Insufficient data to compute distribution consistency!
Start hour: 0.0, End hour: 23.0, KF cutoff: 1.1399999999999983
Insufficient data to compute distribution consistency!
Start hour: 0.0, End hour: 23.0, KF cutoff: 1.1449999999999982
Insufficient data to compute distribution consistency!
Start hour: 0.0, End hour: 23.0, KF cutoff: 1.14999999999

2021-01-15 15:17:32,715 INFO  compute_filter_task -> {'best_start_hr': 0.0, 'best_end_hr': 23.0, 'best_kf_cutoff': 1.109999999999999}
2021-01-15 15:17:32,716 INFO  compute_filter_task -> sidtest3/vikane_pen_id_60_2020-08-10_2020-08-30/sampling_filter.json


Start hour: 0.0, End hour: 23.0, KF cutoff: 1.249999999999996
Insufficient data to compute distribution consistency!
Insufficient data to compute distribution consistency!
Insufficient data to compute distribution consistency!
2020-08-07 2020-08-13
['2020-08-07', '2020-08-08', '2020-08-09', '2020-08-10', '2020-08-11', '2020-08-12', '2020-08-13']
2020-08-07 2020-08-13
['2020-08-07', '2020-08-08', '2020-08-09', '2020-08-10', '2020-08-11', '2020-08-12', '2020-08-13']
2020-08-07 2020-08-13
['2020-08-07', '2020-08-08', '2020-08-09', '2020-08-10', '2020-08-11', '2020-08-12', '2020-08-13']
2020-08-07 2020-08-13
['2020-08-07', '2020-08-08', '2020-08-09', '2020-08-10', '2020-08-11', '2020-08-12', '2020-08-13']
2020-08-08 2020-08-14
['2020-08-08', '2020-08-09', '2020-08-10', '2020-08-11', '2020-08-12', '2020-08-13', '2020-08-14']
2020-08-08 2020-08-14
['2020-08-08', '2020-08-09', '2020-08-10', '2020-08-11', '2020-08-12', '2020-08-13', '2020-08-14']
2020-08-08 2020-08-14
['2020-08-08', '2020-08-0

2021-01-15 15:17:34,965 INFO  generate_report_task -> /tmp/batches/sidtest3/vikane_pen_id_60_2020-08-10_2020-08-30/report.csv


2020-08-22 2020-08-28
['2020-08-22', '2020-08-23', '2020-08-24', '2020-08-25', '2020-08-26', '2020-08-27', '2020-08-28']
2020-08-22 2020-08-28
['2020-08-22', '2020-08-23', '2020-08-24', '2020-08-25', '2020-08-26', '2020-08-27', '2020-08-28']
2020-08-23 2020-08-29
['2020-08-23', '2020-08-24', '2020-08-25', '2020-08-26', '2020-08-27', '2020-08-28', '2020-08-29']
2020-08-23 2020-08-29
['2020-08-23', '2020-08-24', '2020-08-25', '2020-08-26', '2020-08-27', '2020-08-28', '2020-08-29']
2020-08-23 2020-08-29
['2020-08-23', '2020-08-24', '2020-08-25', '2020-08-26', '2020-08-27', '2020-08-28', '2020-08-29']
2020-08-23 2020-08-29
['2020-08-23', '2020-08-24', '2020-08-25', '2020-08-26', '2020-08-27', '2020-08-28', '2020-08-29']
2020-08-24 2020-08-30
['2020-08-24', '2020-08-25', '2020-08-26', '2020-08-27', '2020-08-28', '2020-08-29', '2020-08-30']
2020-08-24 2020-08-30
['2020-08-24', '2020-08-25', '2020-08-26', '2020-08-27', '2020-08-28', '2020-08-29', '2020-08-30']
2020-08-24 2020-08-30
['2020-08-

In [4]:
import pandas as pd
from pprint import pprint
import numpy as np

DEFAULT_LOSS_FACTOR = 0.17

def backtest_accuracy_data(cohort_names, other_loss_factors=list(np.arange(0.13, 0.19, 0.01))):
    out_rows = []
    for i, cohort_name in enumerate(cohort_names):
        report_path = f'/tmp/batches/{BATCH_NAME}/{cohort_name}/report.csv'
        if os.path.exists(report_path):
            report = pd.read_csv(report_path)
            gt_meta = json.load(open(f'/tmp/batches/{BATCH_NAME}/{cohort_name}/ground_truth_metadata.json'))
            rows = report[report['date'] >= gt_meta['last_feeding_date']]
            
            # Must have data on or day after feeding date
            if len(rows) > 0:
                
                if len(rows)>1:
                    chosen_row = rows.iloc[1]
                else:
                    chosen_row = rows.iloc[0]

                smart_avg = chosen_row['smart_average_weight']

                out = dict()
                out['cohort'] = cohort_name
                if gt_meta['expected_loss_factor'] is None:
                    gt_meta['expected_loss_factor'] = 0.13
                if gt_meta['expected_loss_factor'] > 1:
                    gt_meta['expected_loss_factor'] /= 100

                pred = smart_avg * (1 - gt_meta['expected_loss_factor'])
                

                out[f'error pct'] = (pred - gt_meta['gutted_average_weight']) / gt_meta['gutted_average_weight']
                out['smart_pred'] = pred
                out['gutted_weight'] = gt_meta['gutted_average_weight']
                out['expected_loss_factor'] = gt_meta['expected_loss_factor']


                if other_loss_factors is not None:
                    for loss_factor in other_loss_factors:
                        lf = round(loss_factor, 2)
                        pred = smart_avg * (1 - lf)
                        out[f'loss_factor:{lf} error pct'] = (pred - gt_meta['gutted_average_weight']) / gt_meta['gutted_average_weight']

                for k in ['kpi', 'raw_sample_size', 'smart_sample_size']:
                    out[k] = chosen_row[k]
                out_rows.append(out)
    out_df = pd.DataFrame(out_rows)
    return out_df
        
backtest_accuracy_data(cohort_names)

,cohort,error pct,smart_pred,gutted_weight,expected_loss_factor,loss_factor:0.13 error pct,loss_factor:0.14 error pct,loss_factor:0.15 error pct,loss_factor:0.16 error pct,loss_factor:0.17 error pct,loss_factor:0.18 error pct,kpi,raw_sample_size,smart_sample_size
0,aplavika_pen_id_95_2020-07-10_2020-07-26,-0.033881,4781.320789,4949,0.175,0.018816,0.007105,-0.004605,-0.016316,-0.028026,-0.039737,1.638737,1477,7181
1,bolaks_pen_id_88_2020-02-28_2020-03-10,-0.016670,4503.652832,4580,0.165,0.024548,0.012771,0.000995,-0.010781,-0.022558,-0.034334,1.523585,1090,7230
2,djubawik_pen_id_153_2020-11-10_2020-11-26,-0.091117,5171.545071,5690,0.160,-0.058657,-0.069477,-0.080297,-0.091117,-0.101937,-0.112757,-0.128455,43,563
3,eldviktaren_pen_id_164_2020-09-21_2020-10-08,0.050012,3533.291362,3365,0.160,0.087513,0.075013,0.062512,0.050012,0.037512,0.025012,1.423111,917,7690
4,habranden_pen_id_100_2020-08-10_2020-08-31,0.001059,4512.774678,4508,0.175,0.055662,0.043528,0.031394,0.019260,0.007126,-0.005008,1.714970,1775,11756
5,kjeppevikholmen_pen_id_5_2019-06-18_2019-07-02,-0.030473,3836.420142,3957,0.170,0.016252,0.004571,-0.007110,-0.018791,-0.030473,-0.042154,1.355377,934,3430
6,langoy_pen_id_108_2020-05-07_2020-05-17,0.025346,4737.100788,4620,0.165,0.068325,0.056045,0.043766,0.031486,0.019207,0.006927,1.079511,318,2232
7,leivsethamran_pen_id_165_2020-10-18_2020-11-13,-0.061562,5808.932655,6190,0.130,-0.061562,-0.072348,-0.083135,-0.093922,-0.104708,-0.115495,-9.360978,3,3246
8,movikodden_pen_id_114_2020-11-03_2020-11-25,-0.045623,3741.159216,3920,0.130,-0.045623,-0.056593,-0.067562,-0.078532,-0.089502,-0.100472,1.473303,1001,6036
9,movikodden_pen_id_167_2020-10-13_2020-10-30,-0.091387,3907.037110,4300,0.180,-0.035983,-0.047064,-0.058145,-0.069225,-0.080306,-0.091387,1.128928,361,2255


# GTSF Results

In [9]:
train_test_dataset = pd.read_csv('/root/data/sid/traintest_weight_dataset.csv')

In [10]:
estimates = estimate_weight_task(train_test_dataset)
estimates

2021-01-15 15:38:08,838 INFO  estimate_weight_task data: 180746 rows
180746it [03:10, 949.76it/s] 


,Unnamed: 0,Unnamed: 0.1,annotation,fish_id,weight,kf,camera_metadata,akpd_score,train_mask,val_mask,test_mask,estimated_weight_g,estimated_length_mm,estimated_k_factor
0,0,0,"{'leftCrop': [{'keypointType': 'ADIPOSE_FIN', ...",190607010048_bolaks-mjanes,4976,1.327622,"{'baseline': 0.12693501988129197, 'focalLength...",0.991304,True,False,False,4945.976138,728.712165,1.278156
1,1,1,"{'leftCrop': [{'keypointType': 'ADIPOSE_FIN', ...",190607010048_bolaks-mjanes,4976,1.327622,"{'baseline': 0.12693501988129197, 'focalLength...",0.953481,True,False,False,5319.556594,741.299781,1.305852
2,2,2,"{'leftCrop': [{'keypointType': 'ADIPOSE_FIN', ...",190607010048_bolaks-mjanes,4976,1.327622,"{'baseline': 0.12693501988129197, 'focalLength...",0.986294,True,False,False,5154.142976,730.146641,1.324116
3,3,3,"{'leftCrop': [{'keypointType': 'ADIPOSE_FIN', ...",190607010048_bolaks-mjanes,4976,1.327622,"{'baseline': 0.12693501988129197, 'focalLength...",0.981766,True,False,False,4936.802387,722.975654,1.306396
4,4,4,"{'leftCrop': [{'keypointType': 'ADIPOSE_FIN', ...",190607010048_bolaks-mjanes,4976,1.327622,"{'baseline': 0.12693501988129197, 'focalLength...",0.987670,True,False,False,5293.039083,738.259934,1.315459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180741,180741,180741,"{'leftCrop': [{'keypointType': 'ADIPOSE_FIN', ...",200623-4d371b46-73d2-4bbe-bd3e-e11c39192cf3,855,1.249672,"{'baseline': 0.1273289997783822, 'focalLength'...",0.973298,True,False,False,888.889208,436.169312,1.071229
180742,180742,180742,"{'leftCrop': [{'keypointType': 'ADIPOSE_FIN', ...",200623-4d371b46-73d2-4bbe-bd3e-e11c39192cf3,855,1.249672,"{'baseline': 0.1273289997783822, 'focalLength'...",0.988657,True,False,False,779.825225,422.730826,1.032299
180743,180743,180743,"{'leftCrop': [{'keypointType': 'ADIPOSE_FIN', ...",200623-4d371b46-73d2-4bbe-bd3e-e11c39192cf3,855,1.249672,"{'baseline': 0.1273289997783822, 'focalLength'...",0.980260,True,False,False,942.713395,446.548096,1.058705
180744,180744,180744,"{'leftCrop': [{'keypointType': 'ADIPOSE_FIN', ...",200623-4d371b46-73d2-4bbe-bd3e-e11c39192cf3,855,1.249672,"{'baseline': 0.1273289997783822, 'focalLength'...",0.987468,True,False,False,862.712413,430.675665,1.079978


In [19]:
train = estimates[estimates['train_mask']]
val = estimates[estimates['val_mask']]

def accuracy(df):
    y_train_pred = df['estimated_weight_g']
    y_train = df['weight']
    stats = {
        'mean_absolute_error_pct': 100 * np.mean(np.abs((y_train_pred - y_train) / y_train)),
        'mean_error_pct': 100 * np.mean(y_train_pred - y_train) / np.mean(y_train)
    }
    return stats

res = {
    'train':accuracy(train),
    'val': accuracy(val)
}
pd.DataFrame(res)

,train,val
mean_absolute_error_pct,6.336687,4.213553
mean_error_pct,-0.476093,-1.704208
